## Nearest Neighbors ( 最近邻 )

sklearn.neighbors 提供了无监督和监督的 neighbors-based ( 基于邻居 ) 的学习方法的功能。无监督的最近邻是许多其他学习方法的基础，特别是 manifold learning ( 多学科 ) 和 spectral clustering ( 频谱聚类 ) 。受监督的 neighbors-based ( 基于邻居 ) 的学习有两种风格：具有离散标签的数据 分类，以及具有连续标签的数据的 回归。<br>
最邻近方法的原理是找到与新点最近的距离最接近的预定数量的训练样本，并从这些中预测标签。样本数可以是用户定义的常数（k-最近邻学习），或者基于点的局部密度（基于半径的邻居学习）而变化。一般来说，距离可以是 metric measure ：标准欧氏距离是最常见的选择。基于邻居的方法被称为非泛化机器学习方法，因为它们简单地 “remember” 其所有训练数据（可能被转换成诸如 Ball Tree 或 KD Tree 的快速索引结构）。<br>
尽管它的简单性，最近的邻居已经成功地进行了大量的分类和回归问题，包括手写数字或卫星图像场景。作为非参数方法，在决策边界非常不规则的分类情况下，通常是成功的。<br>
sklearn.neighbors 中的类可以处理 Numpy 数组或 scipy.sparse matrices 作为输入。对于密集矩阵，支持大量可能的距离度量。对于稀疏矩阵，搜索可支持任意的 Minkowski metrics 。<br>
有许多依靠最近的邻居的学习习惯是核心的。density estimation ( 密度估计 ) 部分讨论的 核密度估计 的一个例子。<br>

## Unsupervised Nearest Neighbors ( 无监督最近邻 )
NearestNeighbors 实现 unsupervised nearest neighbors learning ( 无监督最近邻学习 ) 。它作为三个不同的最近邻算法的统一接口： BallTree ， KDTree 和基于 sklearn.metrics.pairwise 中的 brute-force algorithm based on routines ( 例程的强力算法 ) 。neighbors search algorithm ( 邻域搜索算法 ) 的选择通过关键字 “algorithm” 进行控制，该算法必须是 ['auto'，'ball_tree'，'kd_tree'，'brute'] 之一。当通过默认值 “auto” 时，算法尝试从训练数据确定最佳方法。有关每个选项的优缺点的讨论

## Finding the Nearest Neighbors ( 找到最近的邻居 )
为了找到两组数据之间最近邻居的简单任务，可以使用  sklearn.neighbors 内的无监督算法：

In [1]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)

In [3]:
distances, indices = nbrs.kneighbors(X)
print(distances)
print(indices)

[[ 0.          1.        ]
 [ 0.          1.        ]
 [ 0.          1.41421356]
 [ 0.          1.        ]
 [ 0.          1.        ]
 [ 0.          1.41421356]]
[[0 1]
 [1 0]
 [2 1]
 [3 4]
 [4 3]
 [5 4]]


## KDTree and BallTree Classes 
或者，可以直接使用 KDTree 或 BallTree 类来查找最近的邻居。这是上面使用的 NearestNeighbors 类所包含的功能。Ball Tree 和 KD Tree 具有相同的界面 ; 我们将在这里展示一个使用 KD Tree 的例子

In [4]:
from sklearn.neighbors import KDTree
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
kdt = KDTree(X, leaf_size=30, metric='euclidean')
kdt.query(X, k=2, return_distance=False)         

array([[0, 1],
       [1, 0],
       [2, 1],
       [3, 4],
       [4, 3],
       [5, 4]])

## Nearest Neighbors Classification ( 最近邻分类 )
Neighbors-based classification ( 基于邻居的分类 ) 是一种 instance-based learning ( 基于实例的学习 ) 或 non-generalizing learning ( 非泛化学习 ) ：它不试图构建一般的内部模型，而只是存储训练数据的实例。分类是从每个点的最近邻居的 simple majority vote ( 简单多数投票 ) 中计算出来的：一个查询点被分配在点的最近邻居中具有最多代表的数据类。<br>
scikit-learn 实现两个不同的最近邻居分类器：KNeighborsClassifier 基于每个查询点的 k 个最近邻居实现学习，其中k是用户指定的整数值。 RadiusNeighborsClassifier 实现基于每个训练点的固定半径 r 内的邻居数量的学习，其中 r 是用户指定的浮点值。<br>
KNeighborsClassifier 中的 k 邻居分类是两种技术中更常用的。值 k 的最佳选择是高度依赖于数据的：通常较大的 k 抑制噪声的影响，但是使得分类边界不太明显。<br>
在数据不均匀采样的情况下，RadiusNeighborsClassifier 中基于半径的邻居分类可以是更好的选择。用户指定一个固定的半径 r ，使得稀疏区域中的点对分类使用较少的最近邻居。对于高维参数空间，由于所谓的 “curse of dimensionality ( 维度惩罚 )” ，该方法变得不太有效。<br>
基本最近邻分类使用均匀权重：即，分配给查询点的值是从最近的邻居的简单多数投票计算的。在某些情况下，最好对邻居进行加权，使邻近邻居更有利于拟合。这可以通过 weights ( 权重 )关键字来实现。默认值 weights = 'uniform' 为每个邻居分配均匀权重。 weight ='distance' 分配与查询点距离的倒数成比例的权重。或者，可以提供用于计算权重的用户定义的距离函数。

## Nearest Neighbor Algorithms ( 最近邻算法 )
### Brute Force
最近邻的快速计算是机器学习研究的一个活跃领域。最简单的 neighbor search ( 邻居搜索 ) 实现涉及数据集中所有点对之间的距离的强力计算：对于 D 维中的 N 个样本，该方法作为  进行比例。对于小数据样本，有效的 brute-force neighbors searches 可能非常有竞争力。然而，随着样本数量 N 的增长，brute-force 迅速变得不可行。在 sklearn.neighbors 中的类中，使用关键字 algorithm ='brute' 指定 brute-force neighbors searches ，并使用 sklearn.metrics.pairwise 中可用的例程来计算。
### K-D Tree
为了解决 brute-force approach ( 强力方法 ) 的计算效率低下，已经发明了各种 tree-based ( 基于树 ) 的数据结构。通常，这些结构试图通过有效地编码样本的总距离信息来减少所需的距离计算数量。基本思想是，如果 A 点距 B 点很远，B 点非常接近 C 点，那么我们知道 A 点和 C 点非常遥远，而不必明确地计算它们的距离。以这种方式，最近邻搜索的计算成本可以减小到  或更好。这是对 brute-force for large N 的显着改善。
利用这种 aggregate information ( 聚合信息 ) 的早期方法是 KD 树数据结构（K 维树的简称），其将二维四叉树和三维十字树概括为任意数量的维度。 KD 树是二进制树结构，它沿着数据轴递归地划分参数空间，将其分割成数据点被归档到的嵌套原点区域。 KD 树的构造非常快：因为仅沿着数据轴执行分割，因此不需要计算 D 维距离。一旦构造，可以仅使用  距离计算来确定查询点的最近邻居。虽然KD树方法对于低维（）邻居搜索来说非常快，但随着 D 增长非常大，它变得无效率：这是所谓的 “curse of dimensionality ( 维度惩罚 )” 的一种表现。在 scikit-learn 中，使用关键字 algorithm ='kd_tree' 指定 KD树 邻居搜索，并使用类 KDTree 计算。
参考：
“Multidimensional binary search trees used for associative searching”, Bentley, J.L., Communications of the ACM (1975)
### Ball Tree
为了解决 KD 树在较高维度上的低效率，开发了 Ball Trees 数据结构。其中 KD 树沿 Cartesian axes ( 笛卡尔轴 ) 分割数据，ball tree 在一系列 nesting hyper-spheres ( 嵌套超球体 ) 中分割数据。这使得树构造比 KD 树的成本更高，但是导致数据结构对于高度结构化的数据可以是非常有效的，即使在非常高的维度上。